In [1]:
import ConstraintsExtraction
from ConstraintsExtraction import *

In [2]:
import ConstraintScalingProjection_LP 
import BregmanProjection
import KernelFDA 
import SC_MPI
import KISSME 
import imp




In [3]:


imp.reload(ConstraintsExtraction),imp.reload(KISSME),imp.reload(BregmanProjection)

(<module 'ConstraintsExtraction' from 'C:\\Users\\49048\\experiments for S2MDC\\ConstraintsExtraction.py'>,
 <module 'KISSME' from 'C:\\Users\\49048\\experiments for S2MDC\\KISSME.py'>,
 <module 'BregmanProjection' from 'C:\\Users\\49048\\experiments for S2MDC\\BregmanProjection.py'>)

In [3]:
def experiments(X,y,proportion_range,random_state_range,gamma=0.1,
                k=1,
                dm=0.1,dc=10,γ=1,
                alpha=1,r=100
               ):
    R_KISSME=KISSME.KISSME_experiments(X,y, 
                            proportion_range=proportion_range,
                            random_state_range=random_state_range,)
    R_BregProj=BregmanProjection.BregmanProjection_experiments(X,y, gamma=gamma,
                        proportion_range=proportion_range,
                        random_state_range=random_state_range,
                        dm=dm,dc=dc,γ=γ,r=r)
    
    
    
    R_SCMPI=SC_MPI.SCMPI_experiments(X,y, gamma=gamma,
                            proportion_range=proportion_range,
                            random_state_range=random_state_range,
                            alpha=alpha)
    R_KFDA=KernelFDA.kernelFDA_experiments(X,y, gamma=gamma,
                            proportion_range=proportion_range,
                            random_state_range=random_state_range)

  
    
    R_CSPLP=ConstraintScalingProjection_LP.ConsScalingProjLP_experiments(X,y, gamma=gamma,
                            proportion_range=proportion_range,
                            random_state_range=random_state_range,
                                 k=1,method='revised simplex')
    
   
    return R_CSPLP,R_BregProj,R_KFDA,R_SCMPI,R_KISSME

    


In [4]:
import matplotlib.pyplot as plt

def plot_comparison(R,criterion=0):
    l=len(R)
    algorithms=['CSP_LP','BregmanProjection','KernelFDA','SCMPI','KISSME']
    criteria=['Kmeans_AMI','Kmeans_ARI','Ward_AMI','Ward_ARI','runtime(s)']
    lines=['-','--','-.','--',':']
    for i in range(l):
        Ri=R[i]
        average_scores=Ri.mean(axis=2)
        var=Ri.var(axis=2)
        plt.errorbar(proportion_range,average_scores[:,criterion],var[:,criterion],
                     label=algorithms[i],linestyle=lines[i],linewidth=2.5,marker='*',markersize=10)
        plt.legend()
        plt.ylabel(ylabel=criteria[criterion],fontsize=18)
        plt.xlabel('percentage of total',fontsize=18)
    
    

# Datasets

In [5]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Lars Buitinck
# License: BSD 3 clause

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# parse commandline arguments
op = OptionParser()
op.add_option(
    "--lsa",
    dest="n_components",
    type="int",
    help="Preprocess documents with latent semantic analysis.",
)
op.add_option(
    "--no-minibatch",
    action="store_false",
    dest="minibatch",
    default=True,
    help="Use ordinary k-means algorithm (in batch mode).",
)
op.add_option(
    "--no-idf",
    action="store_false",
    dest="use_idf",
    default=True,
    help="Disable Inverse Document Frequency feature weighting.",
)
op.add_option(
    "--use-hashing",
    action="store_true",
    default=False,
    help="Use a hashing feature vectorizer",
)
op.add_option(
    "--n-features",
    type=int,
    default=10000,
    help="Maximum number of features (dimensions) to extract from text.",
)
op.add_option(
    "--verbose",
    action="store_true",
    dest="verbose",
    default=False,
    help="Print progress reports inside k-means algorithm.",
)

print(__doc__)


def is_interactive():
    return not hasattr(sys.modules["__main__"], "__file__")


if not is_interactive():
    op.print_help()
    print()

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)

Automatically created module for IPython interactive environment


In [6]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]
# Uncomment the following to do the analysis on all the categories
# categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

dataset = fetch_20newsgroups(
    subset="all", categories=None, shuffle=True, random_state=42
)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
18846 documents
20 categories



In [7]:
labels = dataset.target
true_k = np.unique(labels).shape[0]

print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(
            n_features=opts.n_features,
            stop_words="english",
            alternate_sign=False,
            norm=None,
        )
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(
            n_features=opts.n_features,
            stop_words="english",
            alternate_sign=False,
            norm="l2",
        )
else:
    print('1')
    vectorizer = TfidfVectorizer(
        max_df=0.5,
        max_features=opts.n_features,
        min_df=2,
        stop_words="english",
        use_idf=opts.use_idf,
    )
X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Since LSA/SVD results are not normalized,
    # we redo the normalization to improve the k-means result.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print(
        "Explained variance of the SVD step: {}%".format(int(explained_variance * 100))
    )

    print()

Extracting features from the training dataset using a sparse vectorizer
1
done in 3.625584s
n_samples: 18846, n_features: 10000



In [8]:
X_=X.todense()

In [48]:
X1,y1=randompick(X_,labels,10)
X1.shape

(9778, 10000)

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [50]:
proportion_range=np.arange(0.1,0.36,0.05)
random_state_range=np.arange(7,8)
news_r11=experiments(X1,y1,proportion_range,random_state_range,gamma=0.5,
                dm=0.01,dc=10,γ=1,
                alpha=1
               )

KISSME i 0
KISSME i 1
KISSME i 2
KISSME i 3
KISSME i 4
KISSME i 5
BregmanProj i 0
BregmanProj i 1
BregmanProj i 2
BregmanProj i 3
BregmanProj i 4
BregmanProj i 5
SCMPI i 0
SCMPI i 1
SCMPI i 2
SCMPI i 3
SCMPI i 4
SCMPI i 5
kernelFDA i 0
kernelFDA i 1
kernelFDA i 2
kernelFDA i 3
kernelFDA i 4
kernelFDA i 5
ConsScalingProjLP i 0
ConsScalingProjLP i 1
ConsScalingProjLP i 2
ConsScalingProjLP i 3
ConsScalingProjLP i 4
ConsScalingProjLP i 5


# Results evaluated by AMI

In [57]:
p_range=proportion_range
for criterion in arange(5):
        r=array(news_r11).mean(axis=3)[:,:,0].T
        criteria=['KmAmi','KmAri','WardAmi','WardAri','runtime']
        cat=criteria[criterion]
        r_df=pd.DataFrame(data=np.round(r,5), index=p_range,
                                         columns=['CSP_LP','Bregman','KernelFDA','SC_MPI','KISSME'])
r_df

,CSP_LP,Bregman,KernelFDA,SC_MPI,KISSME
0.10,0.65169,0.59198,0.42405,0.39926,0.68939
0.15,0.70535,0.61737,0.40505,0.42518,0.73565
0.20,0.75781,0.55542,0.40729,0.42510,0.76360
0.25,0.74491,0.56551,0.37800,0.44883,0.78429
0.30,0.81538,0.64201,0.33779,0.47314,0.80773
0.35,0.82967,0.60174,0.38455,0.49310,0.82500


# Time cost

In [58]:
p_range=proportion_range
for criterion in arange(5):
        r=array(news_r11).mean(axis=3)[:,:,4].T
        criteria=['KmAmi','KmAri','WardAmi','WardAri','runtime']
        cat=criteria[criterion]
        r_df=pd.DataFrame(data=np.round(r,5), index=p_range,
                                         columns=['CSP_LP','Bregman','KernelFDA','SC_MPI','KISSME'])
r_df

,CSP_LP,Bregman,KernelFDA,SC_MPI,KISSME
0.10,0.06636,754.16528,657.59280,160.58468,880.94341
0.15,0.04163,974.04653,174.90465,383.54031,891.99594
0.20,0.39190,2742.24916,182.04602,375.48780,180.88584
0.25,0.39701,1281.40659,172.42714,354.91210,198.09551
0.30,0.04999,1562.58511,176.91378,351.00352,153.19213
0.35,0.06806,1699.75825,180.67712,351.17143,170.97177
